In [100]:
import pandas as pd
import numpy as np


# dataset preparation
dataDict = {'District' : ['Suburban', 'Suburban', 'Rural', 'Urban', 'Urban', 'Urban', 'Rural', 'Suburban', 'Suburban', 'Urban', 'Suburban',
                      'Rural', 'Rural', 'Urban'],
        'House Type' : ['Detached', 'Detached', 'Detached', 'Semi-detached', 'Semi-detached', 'Semi-detached', 'Semi-detached', 'Terrace',
                        'Semi-detached', 'Terrace', 'Terrace', 'Terrace', 'Detached', 'Terrace'],
        'Income' : ['High', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'High', 'Low', 'Low', 'Low', 'High', 'Low', 'High'],
        'Previous Customer' : ['No', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes'],
        'Outcome' : ['Not responded', 'Not responded', 'Responded', 'Responded', 'Responded', 'Not responded', 'Responded', 'Not responded',
                     'Responded','Responded','Responded','Responded','Responded', 'Not responded']}

data = pd.DataFrame(dataDict)
columns = ['District', 'House Type', 'Income', 'Previous Customer']
target = 'Outcome'
data

,District,House Type,Income,Previous Customer,Outcome
0,Suburban,Detached,High,No,Not responded
1,Suburban,Detached,High,Yes,Not responded
2,Rural,Detached,High,No,Responded
3,Urban,Semi-detached,High,No,Responded
4,Urban,Semi-detached,Low,No,Responded
5,Urban,Semi-detached,Low,Yes,Not responded
6,Rural,Semi-detached,Low,Yes,Responded
7,Suburban,Terrace,High,No,Not responded
8,Suburban,Semi-detached,Low,No,Responded
9,Urban,Terrace,Low,No,Responded


In [120]:
# function calculating entropy

def entropy(target_col):
    element, count = np.unique(target_col, return_counts=True)
    entropy = np.sum(
        [-(count[i] / np.sum(count)) * np.log2(count[i] / np.sum(count))
         for i in range(len(element))])
    return entropy

# function calculating info gain

def infoGain(data, col, target_col):
    print(col)
    root_entropy = entropy(target_col)
    cols, count = np.unique(data[col], return_counts=True)
    sub = 0
    for i in range(len(cols)):
        sub = sub + count[i] * entropy( data.where(data[col] == cols[i]).dropna()[target_col])
    sub = sub/len(data[col])
    info_gain = root_entropy - sub
    return info_gain


# function bulding tree

def tree_maker(data, features, target_col):
    if len(data[target_col].value_counts()) == 1:
        return data[target_col].value_counts().index[0]
    else:
        info_gains = [infoGain(data, i, target_col) for i in features]
        index = np.argmax(info_gains)
        picked = features[index]
        tree = {picked : {}}
        features = [i for i in features if i != picked]

        for value in np.unique(data[picked]):
            sub_data = data.where(data[picked] == value).dropna()
            sub_tree = tree_maker(sub_data, features, target_col)
            tree[picked][value]= sub_tree
    return tree



In [121]:
tree = tree_maker(data, columns, target)
print(tree)

District
House Type
Income
Previous Customer
House Type
Income
Previous Customer
House Type
Income
Previous Customer
{'District': {'Rural': 'Responded', 'Suburban': {'Income': {'High': 'Not responded', 'Low': 'Responded'}}, 'Urban': {'Previous Customer': {'No': 'Responded', 'Yes': 'Not responded'}}}}
